In [3]:
import pandas as pd
import pymorphy2

from tqdm import tqdm
tqdm.pandas()

import nltk
from nltk.tokenize import word_tokenize 
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
# import eli5

In [4]:
df = pd.read_csv("data.csv", delimiter='\t')
df = df[df['sentiment'] != 'neautral']
df['review_processed'] = df['review'].apply(lambda x: re.sub(r'[^\w\s]', '', x)).values
df['review_processed'] = df['review'].apply(lambda x: word_tokenize(x))

morph = pymorphy2.MorphAnalyzer()

df['review_morphed'] = df['review_processed'].progress_apply(lambda x: [morph.parse(word)[0].normal_form for word in x])


AttributeError: module 'inspect' has no attribute 'getargspec'

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
X = vectorizer.fit_transform([" ".join(x) for x in df['review_morphed']])
X.shape

(60000, 26060)

In [ ]:
train_ds, test_ds, train_marks, test_marks = train_test_split(X, df['sentiment'], test_size=0.3, random_state=42)

reg = LogisticRegression()
reg.fit(train_ds, train_marks)
predict = reg.predict_log_proba(test_ds)[:, 1]

roc_auc_score(test_marks, predict)

0.9770700752068624

In [ ]:
# егор
# bigrams = Phrases(df['review_processed'].values, min_count=2, threshold=1)
# message = "я недавно купил не хороший товар не рекомендую качество плохое не подойти не первый".split()
# print(message)
# print(bigrams.export_phrases())
# print(bigram[message])

In [ ]:
# # они работают вот так
# from gensim.models import Phrases
# documents = ["Нью Йорк большой город", "Нью Йорк", "Поеду в Нью Йорк на следующей неделе"]

# sentence_stream = [doc.split(" ") for doc in documents]
# bigram = Phrases(sentence_stream, min_count=2, threshold=1)
# sent = ['Он', 'поехал', 'в', 'Нью', 'Йорк']
# print(bigram[sent])

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html дока